<a href="https://colab.research.google.com/github/JoaoMiguel-A01/Projeto_VerificaPDF/blob/main/Conhe%C3%A7a_o_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import csv
import json
import re
from datetime import datetime
import calendar
from collections import defaultdict
import xml.etree.ElementTree as ET
import pandas as pd


def associar_pdf_com_xml(pasta_saida, pasta_entrada, caminho_json_xml):
    """
    Associa arquivos PDF aos seus respectivos XMLs com base no número da NF.

    Parâmetros:
    - pasta_saida: Caminho para o diretório que contém os arquivos XML validados.
    - pasta_entrada: Caminho para o diretório que contém os PDFs não validados.
    - caminho_json_xml: Caminho para o arquivo JSON com as informações de validação.
    """

    # Obter lista de arquivos XML e PDFs
    xml_files = [f for f in os.listdir(pasta_saida) if f.endswith('.xml')]
    pdf_files = [f for f in os.listdir(pasta_entrada) if f.endswith('.pdf')]

    # Carregar o JSON com os dados das NFs validadas
    try:
        with open(caminho_json_xml, 'r') as json_file:
            dados_json = json.load(json_file)
            # Criar um dicionário {numero_nf: chave_nf} a partir do JSON
            numero_para_chave_nf = {item['numero_nf']: item['chave_nf'] for item in dados_json}
    except FileNotFoundError:
        print("Erro: Arquivo JSON não encontrado no caminho especificado.")
        return
    except KeyError:
        print("Erro: Estrutura do JSON inválida (faltando 'numero_nf' ou 'chave_nf').")
        return

    for xml_file in xml_files:
        # Extrair o número da NF do nome do arquivo XML (presente no padrão "NF-<numero_nf>")
        try:
            numero_nf_xml = xml_file.split('-')[-1].replace('.xml', '')
        except IndexError:
            print(f"Erro ao extrair o número da NF do XML: {xml_file}")
            continue

        # Verificar se o número da NF está no dicionário do JSON
        if numero_nf_xml in numero_para_chave_nf:
            chave_nf = numero_para_chave_nf[numero_nf_xml]  # Obter a chave de acesso correspondente
            # Procurar PDF correspondente pela chave da NF (prefixo do nome do PDF)
            pdf_correspondente = next((pdf for pdf in pdf_files if pdf.startswith(chave_nf)), None)

            if pdf_correspondente:
                # Renomear o arquivo PDF para ter o mesmo nome do XML
                origem_pdf = os.path.join(pasta_entrada, pdf_correspondente)
                destino_pdf = os.path.join(pasta_saida, f"{xml_file.replace('.xml', '')}.pdf")

                # Verificar se o arquivo PDF existe antes de tentar renomear
                if os.path.exists(origem_pdf):
                    os.rename(origem_pdf, destino_pdf)
                    print(f"Associado: {xml_file} ↔ {pdf_correspondente}")
                else:
                    print(f"Erro: Arquivo PDF '{pdf_correspondente}' não encontrado em '{pasta_entrada}'.")
            else:
                print(f"PDF correspondente não encontrado para a chave: {chave_nf}")
        else:
            print(f"Número da NF não encontrado no JSON: {numero_nf_xml}")



# Função para garantir que as pastas existam e informar o usuário
def garantir_e_informar_pastas(pasta_entrda, pasta_saida, pasta_nao_corresponde):
    # Lista de pastas essenciais
    pastas = {
        "Pasta de Entrada (naoValidados)": pasta_entrada,
        "Pasta de Saída (validados)": pasta_saida,
        "Pasta de Saída (naoCorresponde)": pasta_nao_corresponde
    }

    # Verificar e criar pastas
    for nome, caminho in pastas.items():
        if not os.path.exists(caminho):  # Se a pasta não existe
            os.makedirs(caminho)  # Cria a pasta
            print(f"[INFO] {nome} criada no diretório: {os.path.abspath(caminho)}")  # Mensagem para o usuário
        else:
            print(f"[INFO] {nome} já existe no diretório: {os.path.abspath(caminho)}")  # Confirma que já existe

# Função para remover tudo exceto números
def somente_numeros(cnpj):
    return re.sub(r'\D', '', cnpj)

# Função para normalizar os dados
def normalizar_dado(dado):
    if isinstance(dado, str):
        return dado.strip().replace(".", "").replace(",", ".").upper()
    return dado

# Função para limpar e normalizar os nomes
def limpar_nome(nome):
    return re.sub(r"\s+", " ", nome.replace("\n", " ").strip())

from datetime import datetime

def comparar_datas(data_xml, data_planilha, nome_arquivo_xml):
    try:
        # Verificar se a data do XML está vazia
        if not data_xml:
            print(f"[AVISO] A data no XML '{nome_arquivo_xml}' está vazia. Verifique manualmente.")
            return True  # Continua a execução sem validar a data

        # Verificar se a data da planilha está vazia
        if not data_planilha:
            print(f"[ERRO] A data na planilha está vazia. Verifique os registros.")
            return False

        # Converter data do XML
        data_formatada_xml = datetime.strptime(data_xml, "%Y-%m-%d")  # ISO (aaaa-mm-dd)

        # Converter data da planilha com suporte para formatos diferentes
        try:
            data_formatada_planilha = datetime.strptime(data_planilha, "%d/%m/%Y")  # dd/mm/aaaa
        except ValueError:
            # Tentar com formato de ano abreviado (dd/mm/aa)
            data_formatada_planilha = datetime.strptime(data_planilha, "%d/%m/%y")  # dd/mm/aa

        # Comparar as datas convertidas
        return data_formatada_xml == data_formatada_planilha
    except ValueError as e:
        print(f"[ERRO] Não foi possível comparar as datas: '{data_xml}' e '{data_planilha}'. Erro: {e}")
        return False  # Retorna falso se houver erro na conversão


# Função para converter a planilha em um arquivo JSON
def converter_planilha_para_json(caminho_planilha, caminho_json):
    # Verificar se o arquivo CSV existe no diretório informado
    if not os.path.exists(caminho_planilha):  # Checa se o arquivo existe
        print("[ERRO] O arquivo CSV especificado não existe no diretório informado.")
        print("[DICA] Verifique se o diretório está correto, o nome do arquivo está correto, ou se o arquivo realmente existe.")
        print("[INFO] Verifique o problema e tente executar o código novamente.")
        return  # Encerra a execução da função
    dados = []
    with open(caminho_planilha, mode="r", encoding="latin-1") as arquivo_csv:
        leitor = csv.reader(arquivo_csv, delimiter=";")
        for i, linha in enumerate(leitor, start=1):
            if len(linha) < 18:  # Verificar se há colunas suficientes na linha
                print(f"[ERRO] Linha {i} ignorada: número insuficiente de colunas ({len(linha)}).")
                continue
            try:
                #NF Compra - para cvs com 28 colunas (ainda sem considerar colunas vazias)
                if len(linha) == 28:  # Verifica o número de colunas do CSV
                #Se o CSV possuir 28 colunas, os dados serão classificados como referentes a uma Nota Fiscal de compra.
                    dados.append({
                        "N_PFE": normalizar_dado(linha[0]),
                        "nome_emissor": limpar_nome(linha[2]),
                        "uf_emissor": normalizar_dado(linha[3]),
                        "cnpj_emissor": normalizar_dado(linha[4]),
                        "nome_destinatario": limpar_nome(linha[5]),
                        "uf_destinatario": normalizar_dado(linha[6]),
                        "cnpj_destinatario": normalizar_dado(linha[7]),
                        "data_vencimento": normalizar_dado(linha[13]),
                        "quant": normalizar_dado(linha[14]),
                        "preco_unitario": normalizar_dado(linha[18]),
                        "valor_total": normalizar_dado(linha[26]),
                        "cfop": normalizar_dado(linha[27])
                    })
                #NF Compra - para cvs com 22 colunas (ainda sem considerar colunas vazias)
                elif len(linha) == 22:
                #Se o CSV possuir 22 colunas, os dados serão classificados como referentes a uma Nota Fiscal de compra.
                    dados.append({
                        "N_PFE": normalizar_dado(linha[0]),
                        "nome_emissor": limpar_nome(linha[2]),
                        "uf_emissor": normalizar_dado(linha[3]),
                        "cnpj_emissor": normalizar_dado(linha[4]),
                        "nome_destinatario": limpar_nome(linha[5]),
                        "uf_destinatario": normalizar_dado(linha[6]),
                        "cnpj_destinatario": normalizar_dado(linha[7]),
                        "data_vencimento": normalizar_dado(linha[12]),
                        "quant": normalizar_dado(linha[13]),
                        "preco_unitario": normalizar_dado(linha[15]),
                        "valor_total": normalizar_dado(linha[20]),
                        "cfop": normalizar_dado(linha[21])
                    })
                #NF Venda - para cvs com 29 colunas (ainda sem considerar colunas vazias)
                elif len(linha) == 29:
                #Se o CSV possuir 29 colunas, os dados serão classificados como referentes a uma Nota Fiscal de venda.
                    dados.append({
                        "N_PFE": normalizar_dado(linha[0]),
                        "nome_emissor": limpar_nome(linha[2]),
                        "uf_emissor": normalizar_dado(linha[3]),
                        "cnpj_emissor": normalizar_dado(linha[4]),
                        "nome_destinatario": limpar_nome(linha[5]),
                        "uf_destinatario": normalizar_dado(linha[6]),
                        "cnpj_destinatario": normalizar_dado(linha[7]),
                        "data_vencimento": normalizar_dado(linha[13]),
                        "quant": normalizar_dado(linha[14]),
                        "preco_unitario": normalizar_dado(linha[18]),
                        "valor_total": normalizar_dado(linha[27]),
                        "cfop": normalizar_dado(linha[28])
                    })
                #NF Venda - para cvs com 21 colunas (ainda sem considerar colunas vazias)
                else:
                    # Mantém o processamento considerando dados de NF de venda.
                    dados.append({
                        "N_PFE": normalizar_dado(linha[0]),
                        "nome_emissor": limpar_nome(linha[2]),
                        "uf_emissor": normalizar_dado(linha[3]),
                        "cnpj_emissor": normalizar_dado(linha[4]),
                        "nome_destinatario": limpar_nome(linha[5]),
                        "uf_destinatario": normalizar_dado(linha[6]),
                        "cnpj_destinatario": normalizar_dado(linha[7]),
                        "data_emissao": normalizar_dado(linha[9]),
                        "data_vencimento": normalizar_dado(linha[11]),
                        "quant": normalizar_dado(linha[12]),
                        "preco_unitario": normalizar_dado(linha[14]),
                        "valor_total": normalizar_dado(linha[19]),
                        "cfop": normalizar_dado(linha[20])
                    })
            except IndexError as e:
                print(f"[ERRO] Linha {i} com erro: {e}. Conteúdo: {linha}")

    # Salvar os dados extraídos da planilha no formato JSON
    with open(caminho_json, "w", encoding="utf-8") as json_file:
        json.dump(dados, json_file, ensure_ascii=False, indent=4)
    print(f"[INFO] Planilha convertida com sucesso. Dados salvos em {caminho_json}.")



# Função para extrair dados do XMLimport xml.etree.ElementTree as ET
def extrair_dados_xml(caminho_xml):
    dados = {
        "arquivo_original": caminho_xml.split("/")[-1],
        "chave_nf": "",
        "nome_emissor": "",
        "data_emissao": "",
        "data_vencimento": "",
        "preco_unitario": "",
        "valor_total": "",
        "uf_destinatário": "",
        "uf_emissor": "",
        "numero_nf": "",
        "cfop": "",
        "quant": "",
        "cnpj_destinatario": "",
        "cnpj_emissor": ""
    }

    print(f"[INFO] Processando arquivo XML: {caminho_xml}")

    try:
        # Ler e interpretar o arquivo XML
        tree = ET.parse(caminho_xml)
        root = tree.getroot()

        # Obter namespace do XML
        namespace = {"nfe": "http://www.portalfiscal.inf.br/nfe"}

        # Capturar Número NF
        numero_nf = root.find(".//nfe:ide/nfe:nNF", namespace)
        if numero_nf is not None:
            dados["numero_nf"] = numero_nf.text.strip()
        else:
            print("[DEBUG] Tag  não encontrada.")

        # Capturar Nome do Emitente
        nome_emissor = root.find(".//nfe:emit/nfe:xNome", namespace)
        if nome_emissor is not None:
            dados["nome_emissor"] = nome_emissor.text.strip()
        else:
            print("[DEBUG] Tag  não encontrada no emitente.")

        # Capturar CNPJ do Emitente
        cnpj_emissor = root.find(".//nfe:emit/nfe:CNPJ", namespace)
        if cnpj_emissor is not None:
            dados["cnpj_emissor"] = cnpj_emissor.text.strip()
        else:
            print("[DEBUG] Tag  não encontrada no emitente.")

        # Capturar Nome do Destinatário
        nome_destinatario = root.find(".//nfe:dest/nfe:xNome", namespace)
        if nome_destinatario is not None:
            dados["nome_destinatario"] = nome_destinatario.text.strip()
        else:
            print("[DEBUG] Tag  não encontrada no destinatário.")

        # Capturar CNPJ do Destinatário
        cnpj_destinatario = root.find(".//nfe:dest/nfe:CNPJ", namespace)
        if cnpj_destinatario is not None:
            dados["cnpj_destinatario"] = cnpj_destinatario.text.strip()
        else:
            print("[DEBUG] Tag  não encontrada no destinatário.")

        #Capturar Data de Emissão
        data_emissao = root.find(".//nfe:ide/nfe:dhEmi", namespace)
        if data_emissao is not None:
            dados["data_emissao"] = data_emissao.text[:10]  # Pega apenas a parte da data (YYYY-MM-DD)
        else:
            print("[DEBUG] Tag  não encontrada.")

        # Capturar Preço Unitário
        preco_unitario = root.find(".//nfe:det/nfe:prod/nfe:vUnCom", namespace)
        if preco_unitario is not None:
            dados["preco_unitario"] = preco_unitario.text.strip()
        else:
            print("[DEBUG] Tag <vUnCom> não encontrada.")

        # Capturar Valor Total
        valor_total = root.find(".//nfe:total/nfe:ICMSTot/nfe:vNF", namespace)
        if valor_total is not None:
            dados["valor_total"] = valor_total.text.strip()
        else:
            print("[DEBUG] Tag  não encontrada.")

        # Capturar UF do Destinatário
        uf = root.find(".//nfe:dest/nfe:enderDest/nfe:UF", namespace)
        if uf is not None:
            dados["uf_destinatário"] = uf.text.strip()
        else:
            print("[DEBUG] Tag  não encontrada no destinatário.")

         # Capturar UF do Emissor
        uf = root.find(".//nfe:emit/nfe:enderEmit/nfe:UF", namespace)
        if uf is not None:
            dados["uf_emissor"] = uf.text.strip()
        else:
            print("[DEBUG] Tag  não encontrada no emissor.")

        # Capturar CFOP
        cfop = root.find(".//nfe:det/nfe:prod/nfe:CFOP", namespace)
        if cfop is not None:
            dados["cfop"] = cfop.text.strip()
        else:
            print("[DEBUG] Tag  não encontrada.")

        # Capturar Quantidade de Produtos
        quant = root.find(".//nfe:det/nfe:prod/nfe:qCom", namespace)
        if quant is not None:
            dados["quant"] = quant.text.strip()
        else:
            print("[DEBUG] Tag  não encontrada.")

        # Capturar Data de Vencimento (se disponível)
        data_vencimento = root.find(".//nfe:cobr/nfe:dup/nfe:dVenc", namespace)
        if data_vencimento is not None:
            dados["data_vencimento"] = data_vencimento.text.strip()
        else:
            print("[DEBUG] Tag  não encontrada.")

        # Capturar a chave da NF
        inf_nfe = root.find(".//nfe:infNFe", namespace)
        if inf_nfe is not None:
            chave_nf = inf_nfe.attrib.get("Id", "")
            if chave_nf.startswith("NFe"):
                dados["chave_nf"] = chave_nf[3:]  # Remove os três primeiros caracteres "NFe"
                print(f"[SUCESSO] Chave NF extraída: {dados['chave_nf']}")
            else:
                print("[DEBUG] A chave NF não começa com 'NFe'.")
        else:
            print("[DEBUG] Tag <infNFe> não encontrada.")

    except ET.ParseError as e:
        print(f"[ERRO] Problema ao processar o XML: {e}")
    except Exception as e:
        print(f"[ERRO] Falha ao processar {caminho_xml}: {e}")

    return dados


# Função para converter os arquivos XML em um arquivo JSON
def converter_xml_para_json(pasta_entrada, caminho_json):
    dados_xml = []
    for nome_arquivo in os.listdir(pasta_entrada):
        if nome_arquivo.endswith(".xml"):
            caminho_xml = os.path.join(pasta_entrada, nome_arquivo)
            dados_xml.append({
                "arquivo_original": nome_arquivo,  # Guarda o nome original do arquivo
                **extrair_dados_xml(caminho_xml)  # Extrai dados e adiciona ao dicionário
            })

    # Salvar os dados extraídos dos XMLs no formato JSON
    with open(caminho_json, "w", encoding="utf-8") as json_file:
        json.dump(dados_xml, json_file, ensure_ascii=False, indent=4)
    print(f"[INFO] XMLs convertidos com sucesso. Dados salvos em {caminho_json}.")

# Função para carregar dados de um arquivo JSON
def carregar_dados_json(caminho_json): # defining the missing function
    with open(caminho_json, 'r', encoding='utf-8') as f:
        return json.load(f)

# Função para validar e renomear arquivos com base nos dados extraídos do XML
def validar_e_renomear_arquivos(caminho_json_planilha, caminho_json_xml, pasta_arquivos, pasta_saida, pasta_nao_validados, caminho_txt_relatorio):
    if not os.listdir(pasta_entrada):  # Retorna uma lista vazia se a pasta estiver vazia
        print("[AVISO] A pasta 'naoValidados' está vazia. Por favor, adicione Notas Fiscais em formato XML e execute novamente.")
        return  # Encerra a execução da função
    # Carregar dados da planilha e do XML em JSON
    dados_planilha = carregar_dados_json(caminho_json_planilha)
    dados_xml = carregar_dados_json(caminho_json_xml)
    nao_validados = []  # Lista para arquivos não validados
    linhas_relatorio = []  # Linhas para o relatório TXT

    # Agrupar registros da planilha por CNPJ
    registros_por_cnpj = defaultdict(list)
    for registro in dados_planilha:
        cnpj_destinatario = somente_numeros(registro.get("cnpj_destinatario", ""))
        cnpj_emissor = somente_numeros(registro.get("cnpj_emissor", ""))
        if cnpj_destinatario == '17221771000101' or cnpj_destinatario == '39953546000100': #Verifica se o CNPJ é da LIASA ou COMEL
            registros_por_cnpj[cnpj_emissor].append(registro)    # NF de compra
        else:
            registros_por_cnpj[cnpj_destinatario].append(registro) # NF de venda

    while dados_xml:
        xml_atual = dados_xml.pop(0)  # Pega o primeiro registro XML
        arquivo_original = xml_atual.get("arquivo_original", None)
        if not arquivo_original:
            mensagem = f"[AVISO] Arquivo original ausente no registro do XML: {xml_atual}"
            print(mensagem)
            linhas_relatorio.append(mensagem)
            nao_validados.append(xml_atual)
            continue
        if cnpj_destinatario == '17221771000101' or cnpj_destinatario == '39953546000100': #Verifica se o CNPJ é da LIASA ou COMEL
            # NF de compra
            cnpj_xml = somente_numeros(xml_atual.get("cnpj_emissor", ""))
            registros_cnpj = registros_por_cnpj.get(cnpj_xml, [])
        else:
            # NF de venda
            cnpj_xml = somente_numeros(xml_atual.get("cnpj_destinatario", ""))
            registros_cnpj = registros_por_cnpj.get(cnpj_xml, [])

        if not registros_cnpj:
            mensagem = f"[FALHA] Nenhum registro correspondente ao CNPJ do XML: {arquivo_original}"
            print(mensagem)
            linhas_relatorio.append(mensagem)
            nao_validados.append(xml_atual)
            continue

        validado = False
        melhor_registro = None
        menor_diferenca = float("inf")  # Para encontrar o registro mais próximo

        # Comparar o XML com os registros do mesmo CNPJ
        for registro_planilha in registros_cnpj:
            # Comparar os campos
            diferencas = [
                abs(float(xml_atual.get("valor_total", 0)) - float(registro_planilha.get("valor_total", 0))) <= 0.01,  # Diferença mínima
                abs(float(xml_atual.get("preco_unitario", 0)) - float(registro_planilha.get("preco_unitario", 0))) <= 0.01,  # Diferença mínima
                float(xml_atual.get("quant", 0)) == float(registro_planilha.get("quant", 0)),
                comparar_datas(xml_atual.get("data_vencimento"), registro_planilha.get("data_vencimento"), arquivo_original),
                xml_atual.get("cfop") == registro_planilha.get("cfop"),
                xml_atual.get("uf_destinatario") == registro_planilha.get("uf_destinatario"),
                xml_atual.get("uf_emissor") == registro_planilha.get("uf_emissor")
            ]


        # Priorizar o registro com menos divergências
        total_diferencas = sum(1 for diff in diferencas if not diff)
        if total_diferencas < menor_diferenca:
            menor_diferenca = total_diferencas
            melhor_registro = registro_planilha

        # Validar com o melhor registro
        if melhor_registro and all(diferencas):  # Exige que TODOS os campos sejam válidos
            validado = True
        else:
            validado = False

        # Validar a data de vencimento, ignorando se estiver vazia
        data_vencimento = xml_atual.get("data_vencimento", "").strip()
        if not data_vencimento:
            print(f"[AVISO] A data de vencimento está vazia no arquivo XML '{arquivo_original}'. Verifique manualmente.")
            mes_alfanumerico = "SEM-MES"  # Substituto para o caso de ausência de data
            mes_referencia = "SEM-REF"
        else:
            try:
                data_vencimento = datetime.strptime(data_vencimento, "%Y-%m-%d")  # ISO (aaaa-mm-dd)
                mes_alfanumerico = f"{data_vencimento.month:02}{str(data_vencimento.year)[-2:]}"
                mes_referencia = calendar.month_abbr[data_vencimento.month - 1].upper() + str(data_vencimento.year)[-2:]
            except ValueError as e:
                print(f"[ERRO] Data de vencimento inválida no arquivo XML '{arquivo_original}': {e}")
                mes_alfanumerico = "DATA-ERRO"
                mes_referencia = "DATA-ERRO"

        # Construir novo nome, mesmo em caso de erro e diferenciar NF de compra/venda
        cnpj_destinatario = somente_numeros(xml_atual.get("cnpj_destinatario", ""))
        if cnpj_destinatario == '17221771000101' or cnpj_destinatario == '39953546000100':
            # NF de compra (LIASA/COMEL como destinatários)
            novo_nome = f"{mes_alfanumerico}_{melhor_registro['nome_emissor']}_{mes_referencia}_NF-{xml_atual.get('numero_nf', 'SEM-NF')}.xml"
        else:
            # NF de venda ou outros casos
            novo_nome = f"{mes_alfanumerico}_{melhor_registro['nome_destinatario']}_{mes_referencia}_NF-{xml_atual.get('numero_nf', 'SEM-NF')}.xml"



        # Renomear e mover o arquivo
        caminho_origem = os.path.join(pasta_arquivos, arquivo_original)
        caminho_destino = os.path.join(pasta_saida, novo_nome)
        print(f"[DEBUG] Tentando mover arquivo de {caminho_origem} para {caminho_destino}")
        if os.path.exists(caminho_origem):
            os.rename(caminho_origem, caminho_destino)
            print(f"[VALIDADO] Arquivo renomeado para: {novo_nome}")
        else:
            mensagem = f"[ERRO] Arquivo XML não encontrado: {arquivo_original}"
            print(mensagem)
            linhas_relatorio.append(mensagem)

    if not validado:
        mensagem = f"[FALHA] XML não validado: {arquivo_original}"
        print(mensagem)
        linhas_relatorio.append(mensagem)

        # Log detalhado das divergências
        if melhor_registro:
            detalhes = f"[CONFLITO] Comparado com registro errado: {melhor_registro}"
            linhas_relatorio.append(detalhes)

        nao_validados.append(xml_atual)


    # Gerar arquivo TXT do relatório com mais detalhes relevantes
    with open(caminho_txt_relatorio, "w", encoding="utf-8") as arquivo_relatorio:
        # Escrever mensagens gerais do log
        for linha in linhas_relatorio:
            arquivo_relatorio.write(linha + "\n")

        # Adicionar detalhes específicos das falhas
        arquivo_relatorio.write("\n=== Relatório de XMLs Não Validados ===\n")
        for xml in nao_validados:
            arquivo_relatorio.write(f"\n[FALHA] XML não validado: {xml.get('arquivo_original', 'Arquivo desconhecido')}\n")
            print(f"[FALHA] Comparação falhou para o XML: {xml.get('arquivo_original', 'Arquivo desconhecido')}")  # Exibir no console

            melhor_registro = xml.get("melhor_registro")
            if melhor_registro:
                arquivo_relatorio.write("    Melhor registro encontrado para comparação:\n")
                arquivo_relatorio.write(f"      - Nome Destinatário: {melhor_registro.get('nome_destinatario', 'Não encontrado')}\n")
                arquivo_relatorio.write(f"      - CNPJ Destinatário: {melhor_registro.get('cnpj_destinatario', 'Não encontrado')}\n")
                arquivo_relatorio.write(f"      - Valor Total: {melhor_registro.get('valor_total', 'Não encontrado')}\n")

                print(f"    Comparado com o registro da planilha: {melhor_registro}")  # Exibir no console

                # Detalhar campos divergentes
                divergencias = []
                for campo, valor_xml in xml.get("diferencas", {}).items():
                    valor_planilha = melhor_registro.get(campo, "Não encontrado")
                    if valor_xml != valor_planilha:
                        divergencias.append(f"    - {campo}: XML='{valor_xml}', Planilha='{valor_planilha}'")
                        print(f"    Divergência no campo '{campo}': XML='{valor_xml}', Planilha='{valor_planilha}'")  # Exibir no console

                if divergencias:
                    arquivo_relatorio.write("    Detalhes das divergências:\n")
                    for divergencia in divergencias:
                        arquivo_relatorio.write(divergencia + "\n")
                else:
                    arquivo_relatorio.write("    Nenhuma divergência significativa encontrada.\n")
            else:
                arquivo_relatorio.write("    Nenhum registro correspondente encontrado na planilha.\n")
                print("    Nenhum registro correspondente encontrado na planilha.")  # Exibir no console

    # Mover XMLs não validados para a pasta separada
    for xml in nao_validados:
        arquivo_origem = os.path.join(pasta_arquivos, xml.get("arquivo_original", ""))
        if os.path.exists(arquivo_origem):
            destino_nao_validado = os.path.join(pasta_nao_validados, xml.get("arquivo_original", ""))
            try:
                os.rename(arquivo_origem, destino_nao_validado)
                print(f"[NAO VALIDADO] Arquivo movido para a pasta 'naoValidados': {destino_nao_validado}")
            except Exception as e:
                print(f"[ERRO] Não foi possível mover o arquivo '{arquivo_origem}' para '{destino_nao_validado}': {e}")
        else:
            mensagem = f"[ERRO] Arquivo não encontrado para mover: {arquivo_origem}"
            print(mensagem)
            linhas_relatorio.append(mensagem)

# Configuração principal
pasta_entrada = r"C:\Users\joaoconstancio\Desktop\VisualStudio\IdentificaPDF\naoValidados"  #Pasta para arquvios nao validados (pasta de entrada)
pasta_saida = r"C:\Users\joaoconstancio\Desktop\VisualStudio\IdentificaPDF\validados"   #Pasta para arquvios validados (pasta de saida para arquivos renomeados/validados)
pasta_nao_corresponde = r"C:\Users\joaoconstancio\Desktop\VisualStudio\IdentificaPDF\naoCorresponde"  #Pasta para arquvios que não correspondem com os dados da planilha (pasta de saida para arquivos que não corresponderam com a planilha)
caminho_planilha = r"C:\Users\joaoconstancio\Desktop\VisualStudio\IdentificaPDF\TestePDFCompra-COMEL.csv" #Caminho para o arquvio csv que será usado como base.
caminho_json_planilha = r"C:\Users\joaoconstancio\Desktop\VisualStudio\IdentificaPDF\dados_planilha.json" #Altere de acordo com o diretório onde o código for executado.
caminho_json_xml = r"C:\Users\joaoconstancio\Desktop\VisualStudio\IdentificaPDF\dados_xml.json"  #Altere de acordo com o diretório onde o código for executado.
caminho_txt_relatorio = r"C:\Users\joaoconstancio\Desktop\VisualStudio\IdentificaPDF\relatorio.txt" #Altere de acordo com o diretório onde o código for executado.

# Garantir que as pastas existam
#os.makedirs(pasta_xml, exist_ok=True)
#os.makedirs(pasta_saida, exist_ok=True)
#os.makedirs(pasta_nao_corresponde, exist_ok=True)

# Chamadas das funções principais
garantir_e_informar_pastas(pasta_entrada, pasta_saida, pasta_nao_corresponde)
converter_planilha_para_json(caminho_planilha, caminho_json_planilha)
converter_xml_para_json(pasta_entrada, caminho_json_xml)  # Atualizado para converter XML em JSON
validar_e_renomear_arquivos(caminho_json_planilha, caminho_json_xml, pasta_entrada, pasta_saida, pasta_nao_corresponde, caminho_txt_relatorio)  # Nova função para XML
associar_pdf_com_xml(pasta_saida, pasta_entrada, caminho_json_xml)




# Diretórios úteis LIASA
#J:\FATURAS\2025\CESSAO
#J:\FATURAS\2025\COMPRA
#COMEL
#J:\COMEL\FATURAS\2025\COMPRAS
#J:\COMEL\FATURAS\2025\VENDAS